In [1]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
from pymongo import GEOSPHERE
import math
import numpy as np
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, features
from folium.plugins import HeatMap, MarkerCluster
from src.functions import *
import warnings
warnings.filterwarnings('ignore')

First, a dataframe is created with all the offices in the mongodb database that are located in:
- Madrid
- Barcelona
- Valencia

In [2]:
df = spain_offices()
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,,39.640730,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,NaN,NaN
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
...,...,...,...,...,...,...
161,FRV,Madrid,1,"C/ Fortuny, 45 bajo izquierda",NaN,NaN
162,FRV,Madrid,1,Rafael Calvo 42,NaN,NaN
163,StreamUK,Madrid,1,c/marathon 6,NaN,NaN
164,AdLINK Group,Madrid,1,affilinet Espana SLU,NaN,NaN


Then, the database is cleaned:
- Missing latidudes and longitudes are filled using known addresses
- If there is neither lat-lon nor address, those offices are dropped
- If the lat-lon is not in one of the cities, those offices are dropped as well

In [3]:
df = clean_offices(df)
df

,name,city,office_number,address,latitude,longitude
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068
1,Modo Marketing,Valencia,1,NaN,39.640730,-0.421944
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465
3,Feasa Valencia,Valencia,1,Cami Vera SN,37.210300,-1.857610
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346
...,...,...,...,...,...,...
156,ticketea,Madrid,1,Marques de Lema,40.445515,-3.706176
158,I6NET,Madrid,1,"C/ Magallanes, 13",40.434287,-3.705676
159,cost4travel,Madrid,1,"Calle Francisco Jose Arroyo, 6",40.452871,-3.614939
160,Genasys,Madrid,1,C/ Ventura de la Vega 5,40.416184,-3.698873


The dataframe is saved for future use

In [ ]:
df.to_csv("Output/spanish_companies.csv")

In [4]:
df = pd.read_csv('Output/spanish_companies.csv')
df = df.set_index('Unnamed: 0')
df.index.name = None

A new dataframe is created with all the company positions and salaries (salary number is taken from glassdoor.com, using average salary in Bacelona for each position)

In [5]:
df2 = company_positions()
df2

,position,number_of_employees,salary
0,Designers,20,30000
1,UI/UX Engineers,5,33000
2,Frontend Developers,10,33000
3,Data Engineers,15,44000
4,Backend Developers,5,39000
5,Account Managers,20,25000
6,Maintenance,1,20000
7,Executives,10,60000
8,CEO,1,80000


Another dataframe is created where all of the requirements for our companies future venue are described and weighted. The weight coluns are as follows:

- Weight: It represents the importance of a particular requirement for a person affected by that requirement. For example, an executive cares about having both a Starbucks and a party place nearby, but he/she cares 4 times more about having a Starbucks, because its weight is 40, compared to the party's place weight, which is 10. Mathematically this means that an executive values having a starbucks 40 meters closer and having a party place 10 meters closer the same.

- Final_weight: For each employee related to a requirement, its salary is multiplied by the weight of the requirement. All of those products are added together, and that sum is the final_weight of the requirement. The reasoning behind using the salary of an employee in the equation is that for the company not each employee has the same value, and a good indicator for such value is an employee's salary.

Some notes:
- This weight column is completely subjective. In a real world scenario it could be filled using surveys or any other more accurate way.
- According to the final_weight, the list of requirements from most important to least important are:
    - Schools
    - Airport
    - Design company
    - Tech startup
    - Party place
    - Starbucks
    - Vegan restaurant
    - Basketball stadium
    - Dog hairdresser

In [6]:
df3 = requirements(df2)
df3

,requirements,related_employees,weight,final_weight
0,Design company,Designers,80,48000000
1,Schools,30% of All,100,94500000
2,Successful tech startup,Frontend Developers/Backend Developers,80,42000000
3,Starbucks,Executives,40,24000000
4,Airport,Account Managers,100,50000000
5,Party,All,10,31500000
6,Vegan restaurant,CEO,300,24000000
7,Basketball stadium,Maintenance,500,10000000
8,Dog hairdresser,All,1,3150000


For each venue a query is made to find the nearest school, starbucks, etc. For each of these requirements, a new column is added to the dataframe and the df is saved as a csv in case the apiused to query stops working for a while.

In [ ]:
df['school_distance'] = get_distance_column("school", df)

In [ ]:
df.to_csv("Output/spanish_companies_school.csv")

In [ ]:
df['starbucks_distance'] = get_distance_column("starbucks", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks.csv")

In [ ]:
df['airport_distance'] = get_distance_column("airport", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport.csv")

In [ ]:
df['vegan_distance'] = get_distance_column("vegan", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan.csv")

In [ ]:
df['basketball_distance'] = get_distance_column("Basketball stadium", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball.csv")

In [ ]:
df['party_distance'] = get_distance_column("party", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party.csv")

In [ ]:
df['dog_distance'] = get_distance_column("pet service", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog.csv")

In [ ]:
df['design_distance'] = get_distance_column("Design", df)

In [ ]:
df.to_csv("Output/spanish_companies_school_starbucks_airport_vegan_basketball_party_dog_design.csv")

In [ ]:
df['tech_distance'] = get_distance_column("tech startup", df)

In [ ]:
df.to_csv("Output/spanish_companies_all.csv")

In [7]:
df = pd.read_csv('Output/spanish_companies_all.csv')
df = df.set_index('Unnamed: 0')
df.index.name = None
df

,name,city,office_number,address,latitude,longitude,school_distance,starbucks_distance,airport_distance,vegan_distance,basketball_distance,party_distance,dog_distance,design_distance,tech_distance
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068,102,144,2694,47,2164,764,1142,290,330
1,Modo Marketing,Valencia,1,NaN,39.640730,-0.421944,6721,17226,16778,18016,10499,13436,11722,12235,10444
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465,608,2559,4660,1996,2256,2052,1363,1149,1132
3,Feasa Valencia,Valencia,1,Cami Vera SN,37.210300,-1.857610,2325,100000,21575,9964,52458,69153,64879,64655,52783
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346,84,2031,3528,1985,2961,2120,1305,866,1055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,cost4travel,Madrid,1,"Calle Francisco Jose Arroyo, 6",40.452871,-3.614939,260,2150,2469,6720,5791,970,1962,1600,1580
160,Genasys,Madrid,1,C/ Ventura de la Vega 5,40.416184,-3.698873,1665,210,8533,743,1731,1794,1005,591,115
162,FRV,Madrid,1,Rafael Calvo 42,40.433930,-3.690270,491,293,9068,1254,1571,548,1731,645,842
164,AdLINK Group,Madrid,1,affilinet Espana SLU,40.423070,-3.711930,322,92,8226,384,842,1561,1502,178,389


The columns are then ordered so they are in the same order as the requirements in df2

In [ ]:
df = ordered_df(df)

In [ ]:
df.to_csv("Output/spanish_companies_ordered.csv")

A new column is added, sum_of_distances, which is calculated as follows:
- For each venue, the distance of the venue to each requirement (nearest school, starbucks, etc.) is multiplied by the weight of the requirement (final_weight column of df3), and all of those weighted distances are then added together.

The best venue will be the one that has the lowest sum_of_distances of all the venues in the dataframe

In [8]:
df['sum_of_distances'] = sum_column(df, df3)
df

,name,city,office_number,address,latitude,longitude,school_distance,starbucks_distance,airport_distance,vegan_distance,basketball_distance,party_distance,dog_distance,design_distance,tech_distance,sum_of_distances
0,e-contratos,Valencia,1,"Periodista Azzati, 5. 3, 6B",39.469301,-0.377068,102,144,2694,47,2164,764,1142,290,330,296393500000
1,Modo Marketing,Valencia,1,NaN,39.640730,-0.421944,6721,17226,16778,18016,10499,13436,11722,12235,10444,4472285600000
2,Efebege,Valencia,1,"General Barroso, 66",39.450211,-0.392465,608,2559,4660,1996,2256,2052,1363,1149,1132,739839300000
3,Feasa Valencia,Valencia,1,Cami Vera SN,37.210300,-1.857610,2325,100000,21575,9964,52458,69153,64879,64655,52783,17878017950000
4,DOMODESK,Valencia,1,C/Alqueria de Benlloch 24,39.461268,-0.398346,84,2031,3528,1985,2961,2120,1305,866,1055,649910750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,cost4travel,Madrid,1,"Calle Francisco Jose Arroyo, 6",40.452871,-3.614939,260,2150,2469,6720,5791,970,1962,1600,1580,868803000000
160,Genasys,Madrid,1,C/ Ventura de la Vega 5,40.416184,-3.698873,1665,210,8533,743,1731,1794,1005,591,115,649436250000
162,FRV,Madrid,1,Rafael Calvo 42,40.433930,-3.690270,491,293,9068,1254,1571,548,1731,645,842,608666800000
164,AdLINK Group,Madrid,1,affilinet Espana SLU,40.423070,-3.711930,322,92,8226,384,842,1561,1502,178,389,509182850000


In [ ]:
df.to_csv("Output/spanish_companies_sum_of_distances.csv")

In [51]:
df = pd.read_csv('Output/spanish_companies_sum_of_distances.csv')
df = df.set_index('Unnamed: 0')
df.index.name = None

In [52]:
df["sum_of_distances"].idxmin()

0

The best venue is actually the first on our dataframe, what a coincidence!!! Here's the info of that particular venue:

In [53]:
df.iloc[0]

name                                   e-contratos
city                                      Valencia
office_number                                    1
address                Periodista Azzati, 5. 3, 6B
latitude                                 39.469301
longitude                                -0.377068
design_distance                                290
school_distance                                102
tech_distance                                  330
starbucks_distance                             144
airport_distance                              2694
party_distance                                 764
vegan_distance                                  47
basketball_distance                           2164
dog_distance                                  1142
sum_of_distances                      226006300000
Name: 0, dtype: object

In order to help people visualize the location of this venue, a map of Valencia will be plotted with that venue as the center of the map and every requirement as a marker, with its corresponding distance as a line. In case Valencia is discarded as a potential city for unknown reasons, a map will be plotted for the best candidate in Madrid and Barcelona.

In [54]:
idx_min = df["sum_of_distances"].idxmin()
val_min = df["sum_of_distances"][df['city']=="Valencia"].idxmin()
bcn_min = df["sum_of_distances"][df['city']=="Barcelona"].idxmin()
mad_min = df["sum_of_distances"][df['city']=="Madrid"].idxmin()

All the code required to place all markers and lines on the map is below, but it can be skipped.

In [55]:
venue_location = [df.at[idx_min, "latitude"], df.at[idx_min, "longitude"]]
map_1 = folium.Map(venue_location, zoom_start=17)

In [56]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_1)

In [57]:
design_location = [39.469913, -0.373776]
coordinates = [venue_location, design_location]
distance = df.at[idx_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_1.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_1)

In [58]:
school_location = [39.468421, -0.378141]
coordinates = [venue_location, school_location]
distance = df.at[idx_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_1.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_1)

In [59]:
tech_location = [39.4698, -0.373265]
coordinates = [venue_location, tech_location]
distance = df.at[idx_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_1.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_1)

In [60]:
cafe_location = [39.467075, -0.374612]
coordinates = [venue_location, cafe_location]
distance = df.at[idx_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_1.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_1)

In [61]:
airport_location = [39.491404, -0.473393]
coordinates = [venue_location, airport_location]
distance = df.at[idx_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_1.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_1)

In [62]:
party_location = [39.463429, -0.372434]
coordinates = [venue_location, party_location]
distance = df.at[idx_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_1.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_1)

In [63]:
vegan_location = [39.469071, -0.377547]
coordinates = [venue_location, vegan_location]
distance = df.at[idx_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_1.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_1)

In [64]:
basketball_location = [39.451829, -0.366131]
coordinates = [venue_location, basketball_location]
distance = df.at[idx_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_1.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_1)

In [65]:
dog_location = [39.459688, -0.383472]
coordinates = [venue_location, dog_location]
distance = df.at[idx_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_1.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_1)

Finally, THE map! It's scrollable, and it really shows how every single important requirement is at less than a 5 minutes walk from our venue (with the exception of the airport, which is understandable since there aren't many)

In [66]:
map_1

The same will be done to create a map of the best venue in Barcelona. Its info it the following:

In [67]:
df.loc[bcn_min]

name                             Trovit
city                          Barcelona
office_number                         1
address                Diagonal 601, 9ª
latitude                      41.388769
longitude                      2.129949
design_distance                     176
school_distance                     430
tech_distance                        47
starbucks_distance                   56
airport_distance                  11257
party_distance                      507
vegan_distance                     1445
basketball_distance                1271
dog_distance                        265
sum_of_distances           679446250000
Name: 12, dtype: object

In [68]:
venue_location = [df.at[bcn_min, "latitude"], df.at[bcn_min, "longitude"]]
map_2 = folium.Map(venue_location, zoom_start=17)

In [69]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_2)

In [70]:
design_location = [41.389258, 2.128659]
coordinates = [venue_location, design_location]
distance = df.at[bcn_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_2.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_2)

In [71]:
school_location = [41.39262, 2.129386]
coordinates = [venue_location, school_location]
distance = df.at[bcn_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_2.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_2)

In [72]:
tech_location = [41.388623, 2.130295]
coordinates = [venue_location, tech_location]
distance = df.at[bcn_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_2.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_2)

In [73]:
cafe_location = [41.388763, 2.130817]
coordinates = [venue_location, cafe_location]
distance = df.at[bcn_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_2.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_2)

In [74]:
airport_location = [41.289084, 2.073963]
coordinates = [venue_location, airport_location]
distance = df.at[bcn_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_2.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_2)

In [75]:
party_location = [41.389888, 2.135753]
coordinates = [venue_location, party_location]
distance = df.at[bcn_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_2.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_2)

In [76]:
vegan_location = [41.377077, 2.122568]
coordinates = [venue_location, vegan_location]
distance = df.at[bcn_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_2.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_2)

In [77]:
basketball_location = [41.380163, 2.120141]
coordinates = [venue_location, basketball_location]
distance = df.at[bcn_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_2.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_2)

In [78]:
dog_location = [41.386995, 2.130976]
coordinates = [venue_location, dog_location]
distance = df.at[bcn_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_2.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_2)

In [79]:
map_2

Last but not least, the map of the best venue in Madrid!

In [80]:
df.loc[mad_min]

name                           AdLINK Group
city                                 Madrid
office_number                             1
address                affilinet Espana SLU
latitude                           40.42307
longitude                          -3.71193
design_distance                         178
school_distance                         322
tech_distance                           389
starbucks_distance                       92
airport_distance                       8226
party_distance                         1561
vegan_distance                          384
basketball_distance                     842
dog_distance                           1502
sum_of_distances               540357800000
Name: 164, dtype: object

In [81]:
venue_location = [df.at[mad_min, "latitude"], df.at[mad_min, "longitude"]]
map_3 = folium.Map(venue_location, zoom_start=17)

In [82]:
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_3)

In [83]:
design_location = [40.422644, -3.709948]
coordinates = [venue_location, design_location]
distance = df.at[mad_min,"design_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_3.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_3)

In [84]:
school_location = [40.424884, -3.714747]
coordinates = [venue_location, school_location]
distance = df.at[mad_min,"school_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_3.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_3)

In [85]:
tech_location = [40.423712, -3.707406]
coordinates = [venue_location, tech_location]
distance = df.at[mad_min,"tech_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_3.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Tech Company", icon=tech_icon)
tech_marker.add_to(map_3)

In [86]:
cafe_location = [40.42332, -3.710883]
coordinates = [venue_location, cafe_location]
distance = df.at[mad_min,"starbucks_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_3.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_3)

In [87]:
airport_location = [40.484793, -3.568002]
coordinates = [venue_location, airport_location]
distance = df.at[mad_min,"airport_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_3.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_3)

In [88]:
party_location = [40.432319, -3.698027]
coordinates = [venue_location, party_location]
distance = df.at[mad_min,"party_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_3.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Party", icon=party_icon)
party_marker.add_to(map_3)

In [89]:
vegan_location = [40.424878, -3.715591]
coordinates = [venue_location, vegan_location]
distance = df.at[mad_min,"vegan_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_3.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_3)

In [90]:
basketball_location = [40.430009, -3.708137]
coordinates = [venue_location, basketball_location]
distance = df.at[mad_min,"basketball_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_3.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Basketball Stadium", icon=basketball_icon)
basketball_marker.add_to(map_3)

In [91]:
dog_location = [40.436567, -3.712888]
coordinates = [venue_location, dog_location]
distance = df.at[mad_min,"dog_distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_3.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Hairdresser", icon=dog_icon)
dog_marker.add_to(map_3)

In [92]:
map_3